## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50].lower()))

['what', 'tv', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

tokenizer1 = WordPunctTokenizer()

data_tok = [tokenizer1.tokenize(i.lower()) for i in data]

In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.2558067 ,  3.0422997 ,  0.2496998 ,  1.5345062 , -0.39807555,
       -0.8225896 ,  0.6258101 , -1.3791792 ,  0.04437042, -1.1274815 ,
        0.3895543 , -1.08539   , -2.8994632 ,  1.1497917 , -0.30667993,
        1.1452947 ,  1.6058515 , -2.8088872 , -0.81526226,  5.508117  ,
        0.31063485, -1.1583172 , -2.4817219 ,  1.018219  , -0.7793587 ,
       -2.2556725 , -5.3246417 , -1.3321003 ,  0.3853337 ,  2.0335207 ,
       -1.2053069 ,  0.7138629 ], dtype=float32)

In [9]:
# or query similar words directly. Go play with it!
model.most_similar('money')

[('customers', 0.7742475271224976),
 ('profits', 0.7494139671325684),
 ('cash', 0.7442541122436523),
 ('profit', 0.7183635830879211),
 ('paytm', 0.6925005316734314),
 ('bitcoin', 0.682636559009552),
 ('paypal', 0.6765397191047668),
 ('stock', 0.6694110631942749),
 ('investing', 0.6692131757736206),
 ('taxes', 0.6678221225738525)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [10]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [11]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843312203884125),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [12]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [13]:
# for each word, compute it's vector with model
word_vectors = np.array([np.array(model.get_vector(xi)) for xi in words])

In [14]:
word_vectors.shape

(1000, 100)

In [15]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2)

word_vectors_pca = pca.fit_transform(word_vectors)


scaler = StandardScaler()
word_vectors_pca = scaler.fit_transform(word_vectors_pca)

In [17]:
# np.var(word_vectors_pca_norm[:, 1])

In [18]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [19]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [20]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [21]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca


word_tsne = TSNE(n_components=2, verbose=100).fit_transform(word_vectors)
scaler = StandardScaler()
word_tsne = scaler.fit_transform(word_tsne)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.004s...
[t-SNE] Computed neighbors for 1000 samples in 0.198s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.035s
[t-SNE] Iteration 50: error = 67.5014572, gradient norm = 0.3214593 (50 iterations in 43.579s)
[t-SNE] Iteration 100: error = 68.5136642, gradient norm = 0.3112693 (50 iterations in 43.307s)
[t-SNE] Iteration 150: error = 69.0928955, gradient norm = 0.2821087 (50 iterations in 40.279s)
[t-SNE] Iteration 200: error = 68.3138504, gradient norm = 0.2956477 (50 iterations in 40.347s)
[t-SNE] Iteration 250: error = 69.6787491, gradient norm = 0.2923762 (50 iterations in 43.306s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.678749
[t-SNE] Iteration 300: error = 1.2273206, gradient norm = 0.0028785 (50 iterations in 32.987s)
[t-SNE] Iteration 350: error = 1.1236831, gradient norm = 0

In [22]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1110', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [23]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    phrase = phrase.lower()
    # 2. tokenize phrase
    tokenizer1 = WordPunctTokenizer()
#     tokenizer2 = RegexpTokenizer(r'\w+')
    phrase = tokenizer1.tokenize(phrase)
#     phrase = model.tokenize(phrase)
#     phrase = tokenizer2.tokenize(' '.join(phrase))
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    #print(phrase)
#     d = np.array([np.array(model.get_vector(i)) for i in phrase])
#     #print(d)
#     np.mean(d, axis=1)
    vector = np.zeros([model.vector_size], dtype='float32')
#     #print(len(phrase))
    d = np.ndarray((len(phrase), 100))
    #print(d.shape)
    for i in range(len(phrase)):
        #print(i)
        #print(phrase[i])
        try:
            
            d[i] = model.get_vector(phrase[i])
            #print(model.get_vector(phrase[i]))
        except:
            d[i] = vector
            #print(vector)
            
        
        
    
#     #print(d)
    vector = np.mean(d, axis=0)
    
    # YOUR CODE
    
    return np.array(vector)
        
    

In [24]:
get_phrase_embedding("I'm very sure. This never happened to me before...")

array([ 0.29360649,  0.23250531,  0.06229821,  0.07309077, -0.18970785,
        0.28609508,  0.11894769, -0.05836923,  0.10774754,  0.14120223,
       -0.02361389, -0.21073469, -4.13676924,  0.02747223, -0.11634308,
       -0.10903023, -0.21927353, -0.02205501, -0.44089923, -0.03440423,
        0.08615012, -0.10777523, -0.07355503,  0.14865015,  0.20855977,
       -0.85736922, -0.12136438, -0.25008538,  0.30375731, -0.117872  ,
       -0.0925091 ,  0.04934232, -0.30815569,  0.14904923,  0.05491738,
        0.12134069,  0.03866071, -0.10965985,  0.08336269, -0.02567685,
       -0.94880206,  0.004219  ,  0.20173069, -0.06139723,  0.21811077,
       -0.13389869,  0.16076661, -0.07775854, -0.11758023, -0.014944  ,
       -0.15343276,  0.11950447,  0.16324193, -0.01619715,  0.11458162,
        0.01410077, -0.10992629,  0.10543385,  0.14527591,  0.0897161 ,
        0.04692377,  0.01512886,  0.03374831, -0.10849154,  0.23245731,
       -0.09608069, -0.10729766, -0.10864162, -0.20441723,  0.07

In [26]:
# vector[::10]

In [27]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

AssertionError: 

In [105]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]
chosen_phrases = data

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(i) for i in chosen_phrases])

In [106]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [ ]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 537272 samples in 9.625s...


In [ ]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [32]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [92]:
# data

In [38]:
from scipy.spatial.distance import cosine

In [100]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    phrase_emb = get_phrase_embedding(query)
#     print(phrase_emb)
    cos_mes = np.array([cosine(phrase_emb, i) for i in data_vectors])
#     print(phrase_emb[:10])
    ind = np.argsort(cos_mes)
#     print(cos_mes[ind][:k])
#     print(ind[0])
    return data[ind][:k]
    
#     return <YOUR CODE: top-k lines starting from most similar>

In [87]:
# x = np.array([3.1, 1.3, 2.4])
# ind = np.argpartition(x, 1)
# x[ind]

In [101]:
find_nearest(query="How do i enter the matrix?", k=10)

C:\Users\Frolov.IV\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


348630


array(['How do I get to the dark web?\n',
       'What should I do to enter hollywood?\n',
       'How do I use the Greenify app?\n',
       'What can I do to save the world?\n', 'How do I win this?\n',
       'How do I think out of the box? How do I learn to think out of the box?\n',
       'How do I find the 5th dimension?\n',
       'How do I use the pad in MMA?\n',
       'How do I estimate the competition?\n',
       'What do I do to enter the line of event management?\n'],
      dtype='<U1170')

In [102]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

C:\Users\Frolov.IV\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


348630
How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [103]:
find_nearest(query="How does Trump?", k=10)

C:\Users\Frolov.IV\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


536175


array(['What does Donald Trump think about Israel?\n',
       'What books does Donald Trump like?\n',
       'What does India think of Donald Trump?\n',
       'What does Donald Trump think of India?\n',
       'What does Donald Trump think of China?\n',
       'What does Donald Trump think about Pakistan?\n',
       'What companies does Donald Trump own?\n',
       'What does Dushka Zapata think about Donald Trump?\n',
       'How does it feel to date Ivanka Trump?\n',
       'What does salesforce mean?\n'], dtype='<U1170')

In [104]:
find_nearest(query="Why don't i ask a question myself?", k=10)

C:\Users\Frolov.IV\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


481169


array(["Why don't I get a date?\n",
       "Why do you always answer a question with a question? I don't, or do I?\n",
       "Why can't I ask a question anonymously?\n",
       "Why don't I get a girlfriend?\n",
       "Why don't I have a boyfriend?\n", "I don't have no question?\n",
       "Why can't I take a joke?\n", "Why don't I ever get a girl?\n",
       "Can I ask a girl out that I don't know?\n",
       "Why don't I have a girlfriend?\n"], dtype='<U1170')

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.